In [ ]:
from ipfn import ipfn
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from epx import Job, ModelConfig, SynthPop

# Import Epistemix plotly template for visualization
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

# Estimate Mortality Probabilities based on Demographic Characteristics

In [ ]:
demographic_codes = {
        "female": 0,
        "male":   1,
        "black_female": 20,
        "black_male": 21,
        "white_female": 10,
        "white_male": 11
    }

In [ ]:
lines = ["code,probs\n"]
for demo, code in demographic_codes.items():
    filename = f'data/life_table_{demo}.csv'
    
    """
    Estimate Gompertz mortality parameters from life tables, as in Tai and Noymer, 2017:
        https://u.demog.berkeley.edu/~andrew/papers/tai_noymer_authorfinal.pdf
        https://doi.org/10.1007/s10144-018-0609-6
        
    2010 life table data saved as CSV files in the data directory. Originally from: 
        https://stacks.cdc.gov/view/cdc/26010/cdc_26010_DS1.pdf
    """
    life_table = pd.read_csv(filename, skiprows=1)
    life_table = life_table.loc[life_table["x"] >= 30]
    life_table["M(x)"] = life_table["d(x)"] / life_table["L(x)"]
    life_table["const"] = 1
    weights = np.diag(life_table["d(x)"].to_numpy())
    X = life_table[["const", "x"]].to_numpy()
    y = np.log(life_table["M(x)"])
    alpha, beta = np.linalg.inv(
                X.T @ weights @ X 
            ) @ X.T @ weights @ y
    
    """
    Use learned Gompertz mortality parameters to predict mortality probabilities 
    for each age between 30 and 100.
    """
    probs = np.exp([alpha + beta*x for x in range(30, 101)])
    for prob in probs:
        lines.append(str(code) + "," + str(prob) + "\n")

"""
Save estimated mortality probabilities in a CSV file that will be accessed by the
`read_list_table()` function in FRED.
"""
lines[-1] = lines[-1].strip()
with open("data/mortality_probs.csv", "w") as f:
    f.writelines(lines)

# Assign Educational Attainment Probabilities based on IPF

In [ ]:
"""
Source:  U.S. Census Bureau, Current Population Survey, 2010 Annual Social and Economic Supplement
    https://www.census.gov/data/tables/2010/demo/educational-attainment/cps-detailed-tables.html
    (Table 3)
"""
age_marginals = {
    "No College": {
            "25-34": 15950,
            "35-54": 35484,
            "55+"  : 36734
        },
    "Some College": {
            "25-34": 7752,
            "35-54": 14025,
            "55+"  : 11885
        },
    "Associate's Degree": {
            "25-34": 3903,
            "35-54": 8886,
            "55+"  : 2460
        },
    "Bachelor's Degree": {
            "25-34": 7821,
            "35-54": 13738,
            "55+"  : 8603
        },
    "Advanced Degree": {
            "25-34": 5659,
            "35-54": 12702,
            "55+"  : 11317
        }
}

sex_marginals = {
    "No College": {
            "M": 43597,
            "F": 44570
        },
    "Some College": {
            "M": 15908,
            "F": 17753
        },
    "Associate's Degree": {
            "M": 7662,
            "F": 10597
        },
    "Bachelor's Degree": {
            "M": 14853,
            "F": 15308
        },
    "Advanced Degree": {
            "M": 14304,
            "F": 15374
        }
}

In [ ]:
joint_counts = []
for attainment in age_marginals.keys():
    initial = np.array([[15000, 15000], [15000, 15000], [15000, 15000]])
    xip = np.array([
                age_marginals[attainment]["25-34"],
                age_marginals[attainment]["35-54"],
                age_marginals[attainment]["55+"]
            ])
    xpj = np.array([
                sex_marginals[attainment]["M"],
                sex_marginals[attainment]["F"]
            ])

    aggregates = [xip, xpj]
    dimensions = [[0], [1]]

    IPF = ipfn.ipfn(initial, aggregates, dimensions, convergence_rate=1e-6)
    results = IPF.iteration()
    results = np.round(results).astype(int)
    joint_counts.append(results)
    print(attainment)
    print("===============")
    print(results)
    print()

In [ ]:
joint_probs = np.array(joint_counts) / np.array(joint_counts).sum(axis=0)
joint_probs

In [ ]:
lines = ["code,probs\n"]
code = 0
for sex_idx in [1, 0]:
    for age_idx in [0, 1, 2]:
        for edu_attainment in range(5):
            prob = joint_probs[edu_attainment, age_idx, sex_idx]
            lines.append(f"{code},{np.round(prob, 5)}\n")
        code += 1
        
lines[-1] = lines[-1].strip()

with open("data/education_probs.csv", "w") as f:
    f.writelines(lines)

# Run the retirement model and analyze the results

In [ ]:
retirement_config = ModelConfig(synth_pop=SynthPop("US_2010.v5", 
                                                   locations=["Lenoir_County_NC"]),
                               start_date='2011-01-01',
                               end_date='2031-12-31')

results_dir = "/home/epx/cl-results"

# Configure FRED job
retirement_job = Job(
    "model/main.fred",
    config=[retirement_config],
    key="retirement_savings_job1",
    results_dir=results_dir,
    size="hot",
    # Select FRED version compatible with selected model
    fred_version="11.0.1"
)



In [ ]:
"""
Note, this model takes a few minutes to run.
"""
# Execute job
retirement_job.execute()

import time
# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(retirement_job.status) != 'DONE':
    if str(retirement_job.status) == 'ERROR':
        logs = retirement_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(retirement_job.status)

In [ ]:
savings = retirement_job.results.csv_output("savings.csv")

race_map = {
    -1 : 'Unspecified',
     0 : 'Unknown',
     1 : 'White',
     2 : 'African American',
     3 : 'American Indian',
     4 : 'Alaska Native',
     5 : 'Tribal',
     6 : 'Asian',
     7 : 'Hawaiian Native',
     8 : 'Other Race',
     9 : 'Multiple Race'
}

sex_map = {
    0 : 'Female',
    1 : 'Male',
}

savings_at_65 = savings.loc[savings["age"] == 65]
savings_at_70 = savings.loc[savings["age"] == 70]
savings_at_75 = savings.loc[savings["age"] == 75]

savings_at_65_by_sex  = savings_at_65[["sex", "savings"]].groupby("sex").agg(["count", "sum"]).droplevel(0, axis=1)
savings_at_65_by_race = savings_at_65[["race", "savings"]].groupby("race").agg(["count", "sum"]).droplevel(0, axis=1)
savings_at_70_by_sex  = savings_at_70[["sex", "savings"]].groupby("sex").agg(["count", "sum"]).droplevel(0, axis=1)
savings_at_70_by_race = savings_at_70[["race", "savings"]].groupby("race").agg(["count", "sum"]).droplevel(0, axis=1)
savings_at_75_by_sex  = savings_at_75[["sex", "savings"]].groupby("sex").agg(["count", "sum"]).droplevel(0, axis=1)
savings_at_75_by_race = savings_at_75[["race", "savings"]].groupby("race").agg(["count", "sum"]).droplevel(0, axis=1)

savings_at_65_by_sex["frac"]  = savings_at_65_by_sex["count"]  / savings_at_65_by_sex["count"].sum()
savings_at_65_by_race["frac"] = savings_at_65_by_race["count"] / savings_at_65_by_race["count"].sum()
savings_at_70_by_sex["frac"]  = savings_at_70_by_sex["count"]  / savings_at_70_by_sex["count"].sum()
savings_at_70_by_race["frac"] = savings_at_70_by_race["count"] / savings_at_70_by_race["count"].sum()
savings_at_75_by_sex["frac"]  = savings_at_75_by_sex["count"]  / savings_at_75_by_sex["count"].sum()
savings_at_75_by_race["frac"] = savings_at_75_by_race["count"] / savings_at_75_by_race["count"].sum()

savings_at_65_by_sex["savings_frac"]  = savings_at_65_by_sex["sum"]  / savings_at_65_by_sex["sum"].sum()
savings_at_65_by_race["savings_frac"] = savings_at_65_by_race["sum"] / savings_at_65_by_race["sum"].sum()
savings_at_70_by_sex["savings_frac"]  = savings_at_70_by_sex["sum"]  / savings_at_70_by_sex["sum"].sum()
savings_at_70_by_race["savings_frac"] = savings_at_70_by_race["sum"] / savings_at_70_by_race["sum"].sum()
savings_at_75_by_sex["savings_frac"]  = savings_at_75_by_sex["sum"]  / savings_at_75_by_sex["sum"].sum()
savings_at_75_by_race["savings_frac"] = savings_at_75_by_race["sum"] / savings_at_75_by_race["sum"].sum()

savings_at_65_by_sex["sex_text"]   = savings_at_65_by_sex.index.map(sex_map)
savings_at_65_by_race["race_text"] = savings_at_65_by_race.index.map(race_map)
savings_at_70_by_sex["sex_text"]   = savings_at_70_by_sex.index.map(sex_map)
savings_at_70_by_race["race_text"] = savings_at_70_by_race.index.map(race_map)
savings_at_75_by_sex["sex_text"]   = savings_at_75_by_sex.index.map(sex_map)
savings_at_75_by_race["race_text"] = savings_at_75_by_race.index.map(race_map) 

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(
     values=savings_at_65_by_race["frac"],
     labels=savings_at_65_by_race["race_text"],
     name="Population Percentage by Race"), 
     row=1, col=1)

fig.add_trace(go.Pie(
     values=savings_at_65_by_race["savings_frac"],
     labels=savings_at_65_by_race["race_text"],
     name="Wealth Percentage by Race"),
    row=1, col=2)

fig.update_layout(
    title="Comparing Population Percentage to Wealth-Owned Percentage at age 65"
)

fig.show()